### we use the modified continuous cartpole env from openai gym of this link

https://gist.github.com/iandanforth/e3ffb67cf3623153e968f2afdfb01dc8

other useful link

https://spinningup.openai.com/en/latest/spinningup/rl_intro3.html#implementing-reward-to-go-policy-gradient

In [118]:
from env import ContinuousCartPoleEnv
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from collections import defaultdict

plt.rcParams['font.size']='14'

env=ContinuousCartPoleEnv()

/Users/wang-j/opt/anaconda3/lib/python3.9/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


In [2]:
env.observation_space

Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)

In [3]:
env.action_space

Box([-1.], [1.], (1,), float32)

In [8]:
env.action_space.sample()

array([-0.45743683], dtype=float32)

In [ ]:
np.random.seed(1)

def Gaussian_policy(theta_mu,theta_sig,s):
    
    mu=theta_mu.T.dot(s)[0]
    upper=theta_sig.T.dot(s)
    sig=np.exp(upper-np.max(upper))[0]
    
    return np.random.normal(mu,sig)[0],mu[0],sig[0]
    
def get_dlog(mu,sig,s,a):
    
    dlog_mu=((a-mu)/(sig**2))*s
    dlog_sig=(((a-mu)**2/sig**2)-1)*s
    
    return dlog_mu,dlog_sig

def get_return(rewards,gm):
    
    R=np.zeros(len(rewards))
    R[-1]=rewards[-1]
    for i in range(2,len(R)+1):
        R[-i]=gm*R[-i+1]+rewards[-i]
        
    return R

def run_reinforce(n_eps=5000,n_stps=1000,gm=0.99,lr=0.0001,baseline=False):
    
    theta_mu=np.zeros((4,1))
    theta_sig=np.zeros((4,1))

    r_all,s_all=[],[]
    for ep in range(n_eps):
        stp,r_sum,done=0,0,False
        states,actions,rewards,mus,sigs=[],[],[],[],[]
        s=env.reset().reshape((4,1))
        for stp in range(n_stps):

            a,mu,sig=Gaussian_policy(theta_mu,theta_sig,s)
            s_,r,done,_=env.step(a)
            s_=s_.reshape((4,1))

            states.append(s)
            actions.append(a)
            rewards.append(r)
            mus.append(mu)
            sigs.append(sig)

            s=s_
            stp+=1
            
            if done:
                break

        R=get_return(rewards,gm)
        
        #update policy parameters
        gmt=1
        for i in range(len(rewards)):
            dlog_mu,dlog_sig=get_dlog(mus[i],sigs[i],states[i],actions[i])
            if baseline:
                theta_mu=theta_mu+lr*gmt*(R[i]-sum(R)/len(R))*dlog_mu
                theta_sig=theta_sig+lr*gmt*(R[i]-sum(R)/len(R))*dlog_sig
            else:
                theta_mu=theta_mu+lr*gmt*(R[i])*dlog_mu
                theta_sig=theta_sig+lr*gmt*(R[i])*dlog_sig
            gmt*=gm

        #if ep%(n_eps//10)==0:
        #    print(f'ep:{ep}, R:{sum(rewards)}, stp:{stp}')

        r_all.append(sum(rewards))
        s_all.append(stp)

    return r_all,s_all

res={}
n_eps=5000
n_runs=10
baselines=[False,True]
labels=['REINFORCE','REINFORCE-baseline']

for i in range(len(labels)):
    for n in range(n_runs):
        print(n)
        r,s=run_reinforce(n_eps=n_eps,baseline=baselines[i])
        res[labels[i]+'_'+str(n)+'_return']=r
        res[labels[i]+'_'+str(n)+'_step']=s

0
1
2


In [199]:
res_df=pd.DataFrame.from_dict(res)
res_df

,REINFORCE_0_return,REINFORCE_0_step,REINFORCE_1_return,REINFORCE_1_step,REINFORCE_2_return,REINFORCE_2_step,REINFORCE_3_return,REINFORCE_3_step,REINFORCE_4_return,REINFORCE_4_step,...,REINFORCE-baseline_5_return,REINFORCE-baseline_5_step,REINFORCE-baseline_6_return,REINFORCE-baseline_6_step,REINFORCE-baseline_7_return,REINFORCE-baseline_7_step,REINFORCE-baseline_8_return,REINFORCE-baseline_8_step,REINFORCE-baseline_9_return,REINFORCE-baseline_9_step
0,20.0,20,18.0,18,8.0,8,12.0,12,6.0,6,...,6.0,6,6.0,6,14.0,14,6.0,6,7.0,7
1,11.0,11,13.0,13,8.0,8,7.0,7,7.0,7,...,10.0,10,6.0,6,20.0,20,8.0,8,18.0,18
2,7.0,7,19.0,19,8.0,8,8.0,8,20.0,20,...,7.0,7,9.0,9,15.0,15,8.0,8,15.0,15
3,7.0,7,11.0,11,9.0,9,8.0,8,7.0,7,...,8.0,8,6.0,6,14.0,14,20.0,20,13.0,13
4,9.0,9,20.0,20,15.0,15,18.0,18,14.0,14,...,12.0,12,13.0,13,20.0,20,6.0,6,11.0,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,450.0,450,625.0,625,1000.0,1000,957.0,957,329.0,329,...,90.0,90,1000.0,1000,12.0,12,342.0,342,200.0,200
4996,1000.0,1000,1000.0,1000,861.0,861,675.0,675,357.0,357,...,298.0,298,1000.0,1000,14.0,14,662.0,662,1000.0,1000
4997,362.0,362,505.0,505,1000.0,1000,393.0,393,858.0,858,...,48.0,48,1000.0,1000,15.0,15,230.0,230,680.0,680
4998,1000.0,1000,1000.0,1000,1000.0,1000,391.0,391,476.0,476,...,93.0,93,949.0,949,12.0,12,590.0,590,743.0,743


In [ ]:
def plot_results_all(res,labels,n_runs=10,feature='return',length=5000,n_smooth=10):
    
    plt.figure(figsize=(8,6))
    
    for i in range(len(labels)):
        res_df=pd.DataFrame.from_dict(res)
        
        mean=res_df[[labels[i]+'_'+str(n)+'_'+feature for n in range(n_runs)]].groupby(np.arange(length)//n_smooth).mean().mean(axis=1)
        std=res_df[[labels[i]+'_'+str(n)+'_'+feature for n in range(n_runs)]].groupby(np.arange(length)//n_smooth).mean().std(axis=1)

        plt.plot(mean,label=labels[i],linewidth=3)
        plt.fill_between(range(len(mean)),mean.add(std),mean.subtract(std), alpha=.3)
        
        
    plt.grid()
    ticks=list(range(len(mean)))#500
    labels=[0,int(len(ticks)/2),len(ticks)-1]
    #print(labels)
    plt.xticks([ticks[i] for i in range(len(ticks)) if i in labels],[0,length//2,length])
    plt.xlabel('episodes')
    plt.ylabel(feature)
    
    plt.legend(loc='upper left')
    plt.savefig(feature+'_reinforce_compare_cartpole.png',dpi=350)
    
    
plot_results_all(res,labels,n_runs=5,feature='return',length=n_eps)


In [ ]:
#check variance convergence